In [2]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile, assemble, QuantumRegister, ClassicalRegister, circuit, extensions
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram
from math import gcd
from numpy.random import randint
import pandas as pd
from fractions import Fraction

In [3]:
def ADD_circuit(bits, add):
    """ Creates a Quantum Circuit that increments a register of qubits in the Fourier Space by the parameter add """
    c_add = QuantumCircuit(bits)
    bin_add = bin(add)[2:]
    while len(bin_add) < bits:
        bin_add = '0' + bin_add
    while len(bin_add) > bits:
        bin_add = bin_add[1:]
    if bin_add == bits * '0':
        c_add.x(0)
        c_add.x(0)
        return c_add
    for cbit in range(len(bin_add)):
        if bin_add[cbit] == '1':
            for qbit in range(bits):
                c_add.p( (np.pi/(2**(cbit - qbit))), bits - qbit - 1)
    return c_add 
def ADD(bits, add):   
    c_add = ADD_circuit(bits, add).to_instruction()
    c_add.name = "add {}".format(add)
    return c_add
def CADD(bits, add):
    c_add = ADD_circuit(bits, add)
    c_add.name = "add {}".format(add)
    return c_add.control(num_ctrl_qubits=1).to_instruction()

In [4]:
def test_ADD(n1, n2, bits):
    """ Use QFT, classical_add, and Inverse QFT to add two numbers """
    bn1 = bin(n1)[2:]
    while len(bn1) < bits:
        bn1 = '0' + bn1
    qr = QuantumRegister(bits, "q")
    qc = QuantumCircuit(qr)
    for bit in range(len(bn1)):
        if bn1[bit] == '1':
            qc.x(qr[bits - bit - 1])
    where = []
    for i in range(len(qr)):
        where.append(qr[i])
    qc.append(circuit.library.QFT(num_qubits = bits, do_swaps = False).to_instruction(), where)
    qc.append(ADD(bits, n2), where)
    qc.append(circuit.library.QFT(num_qubits = bits, do_swaps = False).inverse().to_instruction(), where)
    qc.measure_all()
    return qc

In [5]:
q = test_ADD(1, 2, 4)
q.draw()

┌───┐┌──────┐┌────────┐┌───────┐ ░ ┌─┐         
   q_0: ┤ X ├┤0     ├┤0       ├┤0      ├─░─┤M├─────────
        └───┘│      ││        ││       │ ░ └╥┘┌─┐      
   q_1: ─────┤1     ├┤1       ├┤1      ├─░──╫─┤M├──────
             │  qft ││  add 2 ││  iqft │ ░  ║ └╥┘┌─┐   
   q_2: ─────┤2     ├┤2       ├┤2      ├─░──╫──╫─┤M├───
             │      ││        ││       │ ░  ║  ║ └╥┘┌─┐
   q_3: ─────┤3     ├┤3       ├┤3      ├─░──╫──╫──╫─┤M├
             └──────┘└────────┘└───────┘ ░  ║  ║  ║ └╥┘
meas: 4/════════════════════════════════════╩══╩══╩══╩═
                                            0  1  2  3

In [6]:
backend = QasmSimulator()
q_comp = transpile(q, backend)
job = backend.run(q_comp, shots = 10)
result = job.result()
result.get_counts(q_comp)

{'0011': 10}

In [7]:
def MODADD(b, m, bits):
    """ Use the Quantum_Add Gate to add to the register number in the modular base; 
        only works if this number < 2 * m """
    qr = QuantumRegister(bits, 'q')
    vr = QuantumRegister(1, 'v')
    ar = QuantumRegister(1, 'a')
    qc = QuantumCircuit(qr, vr, ar)
    where = []
    for i in range(bits):
        where.append(qr[i])
    where = where + [vr[0]]
    qc.append(ADD(bits + 1, b), where)
    qc.append(ADD(bits + 1, m).inverse(), where)
    qc.append(circuit.library.QFT(num_qubits=bits + 1, do_swaps=False).inverse().to_instruction(), where)
    qc.cx(vr[0],ar[0])
    qc.append(circuit.library.QFT(num_qubits=bits + 1, do_swaps=False).to_instruction(), where)
    where2 = [ar[0]] + where
    qc.append(CADD(bits + 1, m), where2)
    qc.append(ADD(bits + 1, b).inverse(), where)
    qc.append(circuit.library.QFT(num_qubits=bits + 1, do_swaps=False).inverse().to_instruction(), where)
    qc.x(vr[0])
    qc.cx(vr[0],ar[0])
    qc.x(vr[0])
    qc.append(circuit.library.QFT(num_qubits=bits + 1, do_swaps=False).to_instruction(), where)
    qc.append(ADD(bits + 1, b), where)
    return qc

In [8]:
q = MODADD(4, 3, 5)
q.draw()

┌────────┐┌───────────┐┌───────┐     ┌──────┐┌──────────┐┌───────────┐»
q_0: ┤0       ├┤0          ├┤0      ├─────┤0     ├┤1         ├┤0          ├»
     │        ││           ││       │     │      ││          ││           │»
q_1: ┤1       ├┤1          ├┤1      ├─────┤1     ├┤2         ├┤1          ├»
     │        ││           ││       │     │      ││          ││           │»
q_2: ┤2       ├┤2          ├┤2      ├─────┤2     ├┤3         ├┤2          ├»
     │  add 4 ││  add 3_dg ││  iqft │     │  qft ││          ││  add 4_dg │»
q_3: ┤3       ├┤3          ├┤3      ├─────┤3     ├┤4 c_add 3 ├┤3          ├»
     │        ││           ││       │     │      ││          ││           │»
q_4: ┤4       ├┤4          ├┤4      ├─────┤4     ├┤5         ├┤4          ├»
     │        ││           ││       │     │      ││          ││           │»
v_0: ┤5       ├┤5          ├┤5      ├──■──┤5     ├┤6         ├┤5          ├»
     └────────┘└───────────┘└───────┘┌─┴─┐└──────┘│          │└───────────┘»
a_0: ────────────────────────────────┤ X ├────────┤0         ├─────────────»
                                     └───┘        └──────────┘             »
«     ┌───────┐               ┌──────┐┌────────┐
«q_0: ┤0      ├───────────────┤0     ├┤0       ├
«     │       │               │      ││        │
«q_1: ┤1      ├───────────────┤1     ├┤1       ├
«     │       │               │      ││        │
«q_2: ┤2      ├───────────────┤2     ├┤2       ├
«     │  iqft │               │  qft ││  add 4 │
«q_3: ┤3      ├───────────────┤3     ├┤3       ├
«     │       │               │      ││        │
«q_4: ┤4      ├───────────────┤4     ├┤4       ├
«     │       │┌───┐     ┌───┐│      ││        │
«v_0: ┤5      ├┤ X ├──■──┤ X ├┤5     ├┤5       ├
«     └───────┘└───┘┌─┴─┐└───┘└──────┘└────────┘
«a_0: ──────────────┤ X ├───────────────────────
«                   └───┘

In [9]:
def test_MODADD(a, b, m, bits):
    """ Function to test the efficacy of the MODADD"""
    bina = bin(a)[2:]
    while(len(bina) < bits):
        bina = '0' + bina
    qr = QuantumRegister(bits, 'q')
    vr = QuantumRegister(1, 'v')
    ar = QuantumRegister(1, 'a')
    qc = QuantumCircuit(qr, vr, ar)
    for bit in range(len(bina)):
        if bina[bit] == '1':
            qc.x(qr[bits - bit - 1])
    where = []
    for i in range(len(qr)):
        where.append(qr[i])
    where = where + [vr[0]]
    qc.append(circuit.library.QFT(num_qubits=bits + 1, do_swaps=False).to_instruction(), where)
    where2 = where + [ar[0]]
    qc.append(MODADD(b, m, bits).to_instruction(), where2)
    qc.append(circuit.library.QFT(num_qubits=bits + 1, do_swaps=False).inverse().to_instruction(), where)
    qc.measure_all()
    return qc
q = test_MODADD(1,2,15,6)
q.draw()
    

┌───┐┌──────┐┌──────────────┐┌───────┐ ░ ┌─┐                     
   q_0: ┤ X ├┤0     ├┤0             ├┤0      ├─░─┤M├─────────────────────
        └───┘│      ││              ││       │ ░ └╥┘┌─┐                  
   q_1: ─────┤1     ├┤1             ├┤1      ├─░──╫─┤M├──────────────────
             │      ││              ││       │ ░  ║ └╥┘┌─┐               
   q_2: ─────┤2     ├┤2             ├┤2      ├─░──╫──╫─┤M├───────────────
             │      ││              ││       │ ░  ║  ║ └╥┘┌─┐            
   q_3: ─────┤3 qft ├┤3             ├┤3 iqft ├─░──╫──╫──╫─┤M├────────────
             │      ││  circuit-144 ││       │ ░  ║  ║  ║ └╥┘┌─┐         
   q_4: ─────┤4     ├┤4             ├┤4      ├─░──╫──╫──╫──╫─┤M├─────────
             │      ││              ││       │ ░  ║  ║  ║  ║ └╥┘┌─┐      
   q_5: ─────┤5     ├┤5             ├┤5      ├─░──╫──╫──╫──╫──╫─┤M├──────
             │      ││              ││       │ ░  ║  ║  ║  ║  ║ └╥┘┌─┐   
   v_0: ─────┤6     ├┤6             ├┤6      ├─░──╫──╫──╫──╫──╫──╫─┤M├───
             └──────┘│              │└───────┘ ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐
   a_0: ─────────────┤7             ├──────────░──╫──╫──╫──╫──╫──╫──╫─┤M├
                     └──────────────┘          ░  ║  ║  ║  ║  ║  ║  ║ └╥┘
meas: 8/══════════════════════════════════════════╩══╩══╩══╩══╩══╩══╩══╩═
                                                  0  1  2  3  4  5  6  7

In [10]:
q_comp = transpile(q, backend)
job = backend.run(q_comp, shots = 10)
result = job.result()
result.get_counts(q_comp)

{'00000011': 10}

In [11]:
def DCMODADD(b, m, bits):
    """ Doubly Controlled Modular Addition Gate (in the Fourier Space)"""
    qc = MODADD(b, m, bits)
    qc = qc.decompose()
    qc.name = "add {} mod {}".format(b, m)
    return qc.control(num_ctrl_qubits = 2)

In [12]:
qc = DCMODADD(0, 1, 7)
qc.draw()

q72_0: ───────■────────
              │        
q72_1: ───────■────────
       ┌──────┴───────┐
  q_0: ┤0             ├
       │              │
  q_1: ┤1             ├
       │              │
  q_2: ┤2             ├
       │              │
  q_3: ┤3             ├
       │              │
  q_4: ┤4 add 0 mod 1 ├
       │              │
  q_5: ┤5             ├
       │              │
  q_6: ┤6             ├
       │              │
  v_0: ┤7             ├
       │              │
  a_0: ┤8             ├
       └──────────────┘

In [13]:
def AFFMULT(a, m, bits):
    """ Given an x in the computational basis, return ax + the value in the b register in the Fourier Space"""
    cr = QuantumRegister(1, 'c')
    xr = QuantumRegister(bits, 'x')
    br = QuantumRegister(bits, 'b')
    vr = QuantumRegister(1, 'v')
    ar = QuantumRegister(1, 'a')
    qc = QuantumCircuit(cr, xr, br, vr, ar)
    br_list = []
    for i in range(bits):
        br_list.append(br[i])
    br_list.append(vr[0])
    qc.append(circuit.library.QFT(num_qubits=bits + 1, do_swaps=False).to_instruction(), br_list)
    for i in range(bits):
        where = br_list
        where = [cr[0]] + [xr[i]] + br_list + [ar[0]]
        qc.append(DCMODADD(((2 ** i) * a % m), m, bits), where)
    qc.append(circuit.library.QFT(num_qubits=bits + 1, do_swaps=False).inverse().to_instruction(), br_list)
    q = qc.to_instruction()
    q.name = "multiply {} mod {}".format(a, m)
    return q

In [14]:
def test_AFFMULT(a, x, b, m, bits):
    """ Function to test the efficacy of the CMULT Gate"""
    cr = QuantumRegister(1, 'c')
    xr = QuantumRegister(bits, 'x')
    br = QuantumRegister(bits, 'b')
    vr = QuantumRegister(1, 'v')
    ar = QuantumRegister(1, 'a')
    qc = QuantumCircuit(cr, xr, br, vr, ar)
    binx = bin(x)[2:]
    while(len(binx) < bits):
        binx = '0' + binx
    for bit in range(len(binx)):
        if binx[bit] == '1':
            qc.x(xr[bits - bit - 1])
    binb = bin(b)[2:]
    while(len(binb) < bits):
        binb = '0' + binb
    for bit in range(len(binb)):
        if binb[bit] == '1':
            qc.x(br[bits - bit - 1])
    br_list = []
    for i in range(bits):
        br_list.append(br[i])
    qc.x(cr[0])
    qc.append(AFFMULT(a, m, bits).inverse(), range(2 * bits + 3))
    qc.measure_all()
    return qc
qc = test_AFFMULT(7, 6, 3, 15, 4)
qc.draw()

┌───┐┌────────────────────────┐ ░ ┌─┐                              
    c_0: ┤ X ├┤0                       ├─░─┤M├──────────────────────────────
         └───┘│                        │ ░ └╥┘┌─┐                           
    x_0: ─────┤1                       ├─░──╫─┤M├───────────────────────────
         ┌───┐│                        │ ░  ║ └╥┘┌─┐                        
    x_1: ┤ X ├┤2                       ├─░──╫──╫─┤M├────────────────────────
         ├───┤│                        │ ░  ║  ║ └╥┘┌─┐                     
    x_2: ┤ X ├┤3                       ├─░──╫──╫──╫─┤M├─────────────────────
         └───┘│                        │ ░  ║  ║  ║ └╥┘┌─┐                  
    x_3: ─────┤4                       ├─░──╫──╫──╫──╫─┤M├──────────────────
         ┌───┐│                        │ ░  ║  ║  ║  ║ └╥┘┌─┐               
    b_0: ┤ X ├┤5  multiply 7 mod 15_dg ├─░──╫──╫──╫──╫──╫─┤M├───────────────
         ├───┤│                        │ ░  ║  ║  ║  ║  ║ └╥┘┌─┐            
    b_1: ┤ X ├┤6                       ├─░──╫──╫──╫──╫──╫──╫─┤M├────────────
         └───┘│                        │ ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐         
    b_2: ─────┤7                       ├─░──╫──╫──╫──╫──╫──╫──╫─┤M├─────────
              │                        │ ░  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐      
    b_3: ─────┤8                       ├─░──╫──╫──╫──╫──╫──╫──╫──╫─┤M├──────
              │                        │ ░  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐   
    v_0: ─────┤9                       ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├───
              │                        │ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐
    a_0: ─────┤10                      ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├
              └────────────────────────┘ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘
meas: 11/═══════════════════════════════════╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩═
                                            0  1  2  3  4  5  6  7  8  9  10

In [15]:
q_comp = transpile(qc, backend)
job = backend.run(q_comp, shots = 10)
result = job.result()
result.get_counts(q_comp)

{'00011001101': 10}

In [16]:
def CSWAP():
    ar = QuantumRegister(1)
    br = QuantumRegister(1)
    cr = QuantumRegister(1)
    qc = QuantumCircuit(cr, ar, br)
    qc.cx(br[0], ar[0])
    qc.toffoli(cr[0], ar[0], br[0])
    qc.cx(br[0], ar[0])
    return qc
def CSWAPREG(bits):
    ar = QuantumRegister(bits)
    br = QuantumRegister(bits)
    cr = QuantumRegister(1)
    qc = QuantumCircuit(cr, ar, br)
    for i in range(bits):
        qc.append(CSWAP(), [cr[0], ar[i], br[i]])
    q = qc.to_instruction()
    q.name = "c_SWAPREG"
    return q

In [17]:
def test_CSWAPREG(b, a, bits):
    ar = QuantumRegister(bits, 'a')
    br = QuantumRegister(bits, 'b')
    cr = QuantumRegister(1, 'c')
    qc = QuantumCircuit(cr, ar, br)
    qc.x(cr[0])
    bina = bin(a)[2:]
    binb = bin(b)[2:]
    while len(bina) < bits:
        bina = '0' + bina
    while len(binb) < bits:
        binb = '0' + binb
    for bit in range(len(bina)):
        if bina[bit] == '1':
            qc.x(ar[bits - bit - 1])
    for bit in range(len(binb)):
        if binb[bit] == '1':
            qc.x(br[bits - bit - 1])
    qc.append(CSWAPREG(bits), range(2*bits + 1))
    return qc
a = test_CSWAPREG(2, 3, 4)
a.measure_all()
a.draw()

┌───┐┌────────────┐ ░ ┌─┐                        
   c_0: ┤ X ├┤0           ├─░─┤M├────────────────────────
        ├───┤│            │ ░ └╥┘┌─┐                     
   a_0: ┤ X ├┤1           ├─░──╫─┤M├─────────────────────
        ├───┤│            │ ░  ║ └╥┘┌─┐                  
   a_1: ┤ X ├┤2           ├─░──╫──╫─┤M├──────────────────
        └───┘│            │ ░  ║  ║ └╥┘┌─┐               
   a_2: ─────┤3           ├─░──╫──╫──╫─┤M├───────────────
             │            │ ░  ║  ║  ║ └╥┘┌─┐            
   a_3: ─────┤4 c_SWAPREG ├─░──╫──╫──╫──╫─┤M├────────────
             │            │ ░  ║  ║  ║  ║ └╥┘┌─┐         
   b_0: ─────┤5           ├─░──╫──╫──╫──╫──╫─┤M├─────────
        ┌───┐│            │ ░  ║  ║  ║  ║  ║ └╥┘┌─┐      
   b_1: ┤ X ├┤6           ├─░──╫──╫──╫──╫──╫──╫─┤M├──────
        └───┘│            │ ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐   
   b_2: ─────┤7           ├─░──╫──╫──╫──╫──╫──╫──╫─┤M├───
             │            │ ░  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐
   b_3: ─────┤8           ├─░──╫──╫──╫──╫──╫──╫──╫──╫─┤M├
             └────────────┘ ░  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘
meas: 9/═══════════════════════╩══╩══╩══╩══╩══╩══╩══╩══╩═
                               0  1  2  3  4  5  6  7  8

In [18]:
q_comp = transpile(a, backend)
job = backend.run(q_comp, shots = 10)
result = job.result()
result.get_counts(q_comp)

{'001100101': 10}

In [19]:
def gcdExtended(a, b):
    if a == 0 : 
        return b, 0, 1
    gcd, x1, y1 = gcdExtended(b%a, a)
    x = y1 - (b//a) * x1
    y = x1
    return gcd, x, y
def mod_inverse(a, m):
    return gcdExtended(a,m)[1] % m

In [20]:
def CMULT(a, m, bits):
    cr = QuantumRegister(1, 'c')
    xr = QuantumRegister(bits, 'x')
    br = QuantumRegister(bits, 'b')
    vr = QuantumRegister(1, 'v')
    ar = QuantumRegister(1, 'a')
    qc = QuantumCircuit(cr, xr, br, vr, ar)
    qc.append(AFFMULT(a, m, bits), range(3 + 2 * bits))
    qc.append(CSWAPREG(bits), range(1 + 2 * bits))
    qc.append(AFFMULT(mod_inverse(a, m), m, bits).inverse(), range(3 + 2 * bits))
    a = qc.to_instruction()
    a.name = "c_MULT"
    return a


In [21]:
def test_CMULT(x, a, m, bits):
    cr = QuantumRegister(1, 'c')
    xr = QuantumRegister(bits, 'x')
    br = QuantumRegister(bits, 'b')
    vr = QuantumRegister(1, 'v')
    ar = QuantumRegister(1, 'a')
    qc = QuantumCircuit(cr, xr, br, vr, ar)
    binx = bin(x)[2:]
    qc.x(cr[0])
    while len(binx) < bits:
        binx = '0' + binx
    for bit in range(len(binx)):
        if binx[bit] == '1':
            qc.x(xr[bits - bit - 1])
    qc.append(CMULT(a, m, bits), range(3 + 2 * bits))
    qc.measure_all()
    return qc
a = test_CMULT(4, 4, 15, 4)
a.decompose().draw()

┌───────────┐┌─────────────────────┐┌────────────┐»
    c_0: ┤ U3(π,0,π) ├┤0                    ├┤0           ├»
         └───────────┘│                     ││            │»
    x_0: ─────────────┤1                    ├┤1           ├»
                      │                     ││            │»
    x_1: ─────────────┤2                    ├┤2           ├»
         ┌───────────┐│                     ││            │»
    x_2: ┤ U3(π,0,π) ├┤3                    ├┤3           ├»
         └───────────┘│                     ││            │»
    x_3: ─────────────┤4                    ├┤4 c_SWAPREG ├»
                      │                     ││            │»
    b_0: ─────────────┤5  multiply 4 mod 15 ├┤5           ├»
                      │                     ││            │»
    b_1: ─────────────┤6                    ├┤6           ├»
                      │                     ││            │»
    b_2: ─────────────┤7                    ├┤7           ├»
                      │                     ││            │»
    b_3: ─────────────┤8                    ├┤8           ├»
                      │                     │└────────────┘»
    v_0: ─────────────┤9                    ├──────────────»
                      │                     │              »
    a_0: ─────────────┤10                   ├──────────────»
                      └─────────────────────┘              »
meas: 11/══════════════════════════════════════════════════»
                                                           »
«         ┌────────────────────────┐ ░ ┌─┐                              
«    c_0: ┤0                       ├─░─┤M├──────────────────────────────
«         │                        │ ░ └╥┘┌─┐                           
«    x_0: ┤1                       ├─░──╫─┤M├───────────────────────────
«         │                        │ ░  ║ └╥┘┌─┐                        
«    x_1: ┤2                       ├─░──╫──╫─┤M├────────────────────────
«         │                        │ ░  ║  ║ └╥┘┌─┐                     
«    x_2: ┤3                       ├─░──╫──╫──╫─┤M├─────────────────────
«         │                        │ ░  ║  ║  ║ └╥┘┌─┐                  
«    x_3: ┤4                       ├─░──╫──╫──╫──╫─┤M├──────────────────
«         │                        │ ░  ║  ║  ║  ║ └╥┘┌─┐               
«    b_0: ┤5  multiply 4 mod 15_dg ├─░──╫──╫──╫──╫──╫─┤M├───────────────
«         │                        │ ░  ║  ║  ║  ║  ║ └╥┘┌─┐            
«    b_1: ┤6                       ├─░──╫──╫──╫──╫──╫──╫─┤M├────────────
«         │                        │ ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐         
«    b_2: ┤7                       ├─░──╫──╫──╫──╫──╫──╫──╫─┤M├─────────
«         │                        │ ░  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐      
«    b_3: ┤8                       ├─░──╫──╫──╫──╫──╫──╫──╫──╫─┤M├──────
«         │                        │ ░  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐   
«    v_0: ┤9                       ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├───
«         │                        │ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐
«    a_0: ┤10                      ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├
«         └────────────────────────┘ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘
«meas: 11/══════════════════════════════╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩═
«                                       0  1  2  3  4  5  6  7  8  9  10

In [22]:
q_comp = transpile(a, backend)
job = backend.run(q_comp, shots = 10)
result = job.result()
result.get_counts(q_comp)

{'00000000011': 10}

In [23]:
def PCMULT(a, m, power, bits):
    cr = QuantumRegister(1, 'c')
    xr = QuantumRegister(bits, 'x')
    br = QuantumRegister(bits, 'b')
    vr = QuantumRegister(1, 'v')
    ar = QuantumRegister(1, 'a')
    qc = QuantumCircuit(cr, xr, br, vr, ar)
    for i in range(power):
        qc.append(CMULT(a, m, bits), range(2 * bits + 3))
    b = qc.to_instruction()
    b.name = "%i ^ %i mod %i" % (a, power, m)
    return b

In [24]:
def test_PCMULT(x, a, m, power, bits):
    cr = QuantumRegister(1, 'c')
    xr = QuantumRegister(bits, 'x')
    br = QuantumRegister(bits, 'b')
    vr = QuantumRegister(1, 'v')
    ar = QuantumRegister(1, 'a')
    qc = QuantumCircuit(cr, xr, br, vr, ar)
    binx = bin(x)[2:]
    qc.x(cr[0])
    while len(binx) < bits:
        binx = '0' + binx
    for bit in range(len(binx)):
        if binx[bit] == '1':
            qc.x(xr[bits - bit - 1])
    qc.append(PCMULT(a, m, power, bits), range(3 + 2 * bits))
    qc.measure_all()
    return qc
a = test_PCMULT(1, 8, 15, 1, 4)
a.draw()

┌───┐┌────────────────┐ ░ ┌─┐                              
    c_0: ┤ X ├┤0               ├─░─┤M├──────────────────────────────
         ├───┤│                │ ░ └╥┘┌─┐                           
    x_0: ┤ X ├┤1               ├─░──╫─┤M├───────────────────────────
         └───┘│                │ ░  ║ └╥┘┌─┐                        
    x_1: ─────┤2               ├─░──╫──╫─┤M├────────────────────────
              │                │ ░  ║  ║ └╥┘┌─┐                     
    x_2: ─────┤3               ├─░──╫──╫──╫─┤M├─────────────────────
              │                │ ░  ║  ║  ║ └╥┘┌─┐                  
    x_3: ─────┤4               ├─░──╫──╫──╫──╫─┤M├──────────────────
              │                │ ░  ║  ║  ║  ║ └╥┘┌─┐               
    b_0: ─────┤5  8 ^ 1 mod 15 ├─░──╫──╫──╫──╫──╫─┤M├───────────────
              │                │ ░  ║  ║  ║  ║  ║ └╥┘┌─┐            
    b_1: ─────┤6               ├─░──╫──╫──╫──╫──╫──╫─┤M├────────────
              │                │ ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐         
    b_2: ─────┤7               ├─░──╫──╫──╫──╫──╫──╫──╫─┤M├─────────
              │                │ ░  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐      
    b_3: ─────┤8               ├─░──╫──╫──╫──╫──╫──╫──╫──╫─┤M├──────
              │                │ ░  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐   
    v_0: ─────┤9               ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├───
              │                │ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐
    a_0: ─────┤10              ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├
              └────────────────┘ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘
meas: 11/═══════════════════════════╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩═
                                    0  1  2  3  4  5  6  7  8  9  10

In [25]:
q_comp = transpile(a, backend)
job = backend.run(q_comp, shots = 10)
result = job.result()
result.get_counts(q_comp)

{'00000010001': 10}

In [75]:
def SHOR(a, m):
    bits = len(bin(m)) - 2
    cr = QuantumRegister(bits, 'c')
    xr = QuantumRegister(bits, 'x')
    br = QuantumRegister(bits, 'b')
    vr = QuantumRegister(1, 'v')
    ar = QuantumRegister(1, 'a')
    meas = ClassicalRegister(bits)
    qc = QuantumCircuit(cr, xr, br, vr, ar, meas)
    for i in range(bits):
        qc.h(cr[i])
    qc.x(xr[2])
    for i in range(bits):
        where = [cr[i]]
        for j in range(bits):
            where.append(xr[j])
        for j in range(bits):
            where.append(br[j])
        where.append(vr[0])
        where.append(ar[0])
        qc.append(PCMULT(a, m, 2**i,bits), where)
    qc.append(circuit.library.QFT(num_qubits=bits).inverse(), range(bits))
    a = []
    b = []
    for i in range(bits):
        a.append(cr[i])
        b.append(meas[i])
    qc.measure(a, b)
    return qc

In [72]:
def find_denominator(bits, bitstring):
    numerator = 0
    for i in range(bits):
        if bitstring[i] == '1':
            numerator += 2 ** (bits - i - 1)
    raw = Fraction(numerator, 2 ** bits) 
    return raw.denominator

In [76]:
class Counter(dict):
    def __missing__(self, key):
        return 0

In [ ]:
def guess(N, g):
    bits = len(bin(N)) - 2
    q_comp = transpile(SHOR(g, N), backend)
    job = backend.run(q_comp, shots = 1000)
    result = job.result()
    a = result.get_counts(q_comp)
    phases = Counter()
    for bstring, frequencies in a.items():
        phases[find_denominator(bits, bstring)] += frequencies
    return phases

{4: 488, 2: 244, 1: 268}

In [ ]:
def validate(N, g, T):
    if T % 2 == 1:
        return False
    factor_1 = gcd(g ** (T//2) - 1, N)
    factor_2 = gcd(g ** (T//2) + 1, N)
    f1_trivial = False
    f2_trivial = False
    if factor_1 == 1 or factor_1 == N:
        f1_trivial = True
    if factor_2 == 1 or factor_2 == N:
        f2_trivial = True
    if f1_trivial and f2_trivial:
        return False
    elif f1_trivial and (f2_trivial == False):
        return (N//factor_2, factor_2)
    elif (f1_trivial == False) and f2_trivial:
        return (factor_1, N//factor_1)
    else:
        return (factor_1, factor_2)

False

In [109]:
def factor(N):
    used = set()
    while True:
        g = np.random.randint(2, N)
        print("trying {}".format(g))
        if g not in used and gcd(g, N) == 1:
            phase = guess(N, g)
            k = sorted(phase, reverse=True)
            for i in k:
                if validate(N, g, i) is not False:
                    print("{} ^ {} worked".format(g, i))
                    return validate(N, g, i)
                else:
                    print("{} ^ {} did not work".format(g, i))
            set.add(g)

In [110]:
factor(21)

trying 2
